In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Dense # type: ignore
import joblib



In [2]:
# CSV dosyasını yükleme
df = pd.read_csv("Emotion_dataset_train.csv")
print(df)



      Unnamed: 0                                           Sentence    Label
0              0  Allah aşkına bi fırça neden 250 tl neden yani ...   kızgın
1              1  Muavin bir şey içip içmediğimi sormak için ben...  surpriz
2              2  Herkese güzel günler öpüyorum sizler tivitır a...    mutlu
3              3  Ya şu sevgililerin birbirine şifre verme olayı...   kızgın
4              4  Almanya'da 'yok artık' dedirten olay! Nereye p...  surpriz
...          ...                                                ...      ...
3895        3895  Onu görünce ben de ürktüm bir başta sonra yeni...    korku
3896        3896  Gece sabaha kadar 50 tane kabus gördüm nasıl b...    korku
3897        3897  Sustum senden sonra, sessize aldım dilimi. Ne ...    korku
3898        3898  mutsuzum çünkü ders çalışamıyorum odaklanamıyo...    üzgün
3899        3899  Metroda bile çeken telefon evde çekmiyor. İnsa...  surpriz

[3900 rows x 3 columns]


In [3]:
# Öznitelikler ve etiketleri ayırma
X = df['Sentence']
y = df['Label']

In [4]:
unique_labels = df['Label'].unique()
print(unique_labels)


['kızgın' 'surpriz' 'mutlu' 'üzgün' 'korku']


In [5]:
# Veri setini eğitim ve test setlerine ayırma
X_egitim, X_test, y_egitim, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [6]:
# Metin verilerini sayısal vektörlere dönüştürme
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_egitim_tfidf = tfidf_vectorizer.fit_transform(X_egitim)

In [7]:
# Doğrulama veri setini dönüştürme ve indekslerini sıralama
X_doğrulama = X_test
X_doğrulama_tfidf = tfidf_vectorizer.transform(X_doğrulama)
X_doğrulama_tfidf.sort_indices()

In [8]:
# Etiketleri sayısal değerlere dönüştürme
label_encoder = LabelEncoder()
y_egitim_encoded = label_encoder.fit_transform(y_egitim)
y_test_encoded = label_encoder.transform(y_test)

In [9]:
# Sinir ağı modeli oluşturma
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_egitim_tfidf.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(64, activation='sigmoid'))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))


In [10]:
# Modeli derleme
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [11]:
# Eğitim veri setini oluşturma
X_egitim, X_test, y_egitim, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
# Eğitim ve doğrulama veri setlerini oluşturma
X_egitim, X_doğrulama, y_egitim, y_doğrulama = train_test_split(X, y, test_size=0.1, random_state=42)


In [13]:
# Metin verilerini sayısal vektörlere dönüştürme
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_egitim_tfidf = tfidf_vectorizer.fit_transform(X_egitim)
X_doğrulama_tfidf = tfidf_vectorizer.transform(X_doğrulama)

In [14]:
# Seyrek matrislerin indekslerini sırala
X_egitim_tfidf.sort_indices()
X_doğrulama_tfidf.sort_indices()


In [15]:
# Etiketleri sayısal değerlere dönüştürme
label_encoder = LabelEncoder()
y_egitim_encoded = label_encoder.fit_transform(y_egitim)
y_doğrulama_encoded = label_encoder.transform(y_doğrulama)

In [16]:
# Modeli eğitme
model.fit(X_egitim_tfidf, y_egitim_encoded, epochs=9, batch_size=64, validation_data=(X_doğrulama_tfidf, y_doğrulama_encoded))


Epoch 1/9
55/55 [==============================] - 2s 7ms/step - loss: 1.6152 - accuracy: 0.1858 - val_loss: 1.5959 - val_accuracy: 0.3872
Epoch 2/9
55/55 [==============================] - 0s 3ms/step - loss: 1.4246 - accuracy: 0.4160 - val_loss: 1.1467 - val_accuracy: 0.4359
Epoch 3/9
55/55 [==============================] - 0s 3ms/step - loss: 0.9689 - accuracy: 0.6131 - val_loss: 0.8111 - val_accuracy: 0.7795
Epoch 4/9
55/55 [==============================] - 0s 4ms/step - loss: 0.6509 - accuracy: 0.8635 - val_loss: 0.5667 - val_accuracy: 0.9205
Epoch 5/9
55/55 [==============================] - 0s 3ms/step - loss: 0.3578 - accuracy: 0.9744 - val_loss: 0.3197 - val_accuracy: 0.9410
Epoch 6/9
55/55 [==============================] - 0s 3ms/step - loss: 0.1527 - accuracy: 0.9886 - val_loss: 0.2317 - val_accuracy: 0.9564
Epoch 7/9
55/55 [==============================] - 0s 3ms/step - loss: 0.0772 - accuracy: 0.9929 - val_loss: 0.2106 - val_accuracy: 0.9564
Epoch 8/9
55/55 [==========

In [17]:
# Test veri setini TF-IDF vektörlerine dönüştürme
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [18]:
# Seyrek matrisin indekslerini sırala
X_test_tfidf.sort_indices()

In [19]:
# Modelin performansını değerlendirme
loss, accuracy = model.evaluate(X_test_tfidf, y_test_encoded)
print("Modelin doğruluk skoru:", accuracy)


 1/25 [>.............................] - ETA: 0s - loss: 0.2749 - accuracy: 0.9688

25/25 [==============================] - 0s 2ms/step - loss: 0.1310 - accuracy: 0.9782
Modelin doğruluk skoru: 0.978205144405365


In [20]:
# Modeli kaydetme
model.save("model.keras")

In [21]:
# model uzerinde tahmin yapma
y_tahmin = model.predict(X_test_tfidf)
y_tahmin = y_tahmin.argmax(axis=-1)
y_tahmin = label_encoder.inverse_transform(y_tahmin)
print(y_tahmin)
print(y_test)
print(accuracy_score(y_test, y_tahmin))


25/25 [==============================] - 0s 1ms/step
['surpriz' 'surpriz' 'korku' 'üzgün' 'surpriz' 'kızgın' 'kızgın' 'mutlu'
 'üzgün' 'kızgın' 'mutlu' 'üzgün' 'kızgın' 'korku' 'kızgın' 'kızgın'
 'kızgın' 'korku' 'mutlu' 'mutlu' 'korku' 'mutlu' 'surpriz' 'surpriz'
 'surpriz' 'kızgın' 'üzgün' 'mutlu' 'üzgün' 'korku' 'korku' 'mutlu'
 'mutlu' 'üzgün' 'üzgün' 'korku' 'korku' 'kızgın' 'surpriz' 'surpriz'
 'korku' 'surpriz' 'üzgün' 'surpriz' 'mutlu' 'surpriz' 'mutlu' 'surpriz'
 'surpriz' 'korku' 'korku' 'kızgın' 'kızgın' 'kızgın' 'mutlu' 'üzgün'
 'kızgın' 'üzgün' 'üzgün' 'surpriz' 'korku' 'korku' 'kızgın' 'üzgün'
 'kızgın' 'kızgın' 'mutlu' 'üzgün' 'korku' 'surpriz' 'korku' 'mutlu'
 'mutlu' 'mutlu' 'üzgün' 'kızgın' 'surpriz' 'mutlu' 'kızgın' 'üzgün'
 'üzgün' 'surpriz' 'korku' 'mutlu' 'surpriz' 'surpriz' 'korku' 'mutlu'
 'korku' 'üzgün' 'kızgın' 'surpriz' 'kızgın' 'korku' 'üzgün' 'mutlu'
 'korku' 'kızgın' 'mutlu' 'üzgün' 'üzgün' 'korku' 'korku' 'surpriz'
 'kızgın' 'kızgın' 'üzgün' 'kızgın' 'su

In [22]:
#verecegim cumlenin duygusunu tahmin edelim. 
cumle = "cüzdanımı unuttuğum için eve yürüyerek dönmek zorunda kaldım "
cumle_tfidf = tfidf_vectorizer.transform([cumle])
cumle_tfidf.sort_indices()
tahmin = model.predict(cumle_tfidf)
tahmin = tahmin.argmax(axis=-1)
tahmin = label_encoder.inverse_transform(tahmin)
print(tahmin)


1/1 [==============================] - 0s 11ms/step


['surpriz']


In [23]:
import joblib

# TF-IDF vektörleştiriciyi kaydet
joblib.dump(tfidf_vectorizer, "tfidf_vectorizer.pkl")

# Etiket kodlayıcıyı kaydet
joblib.dump(label_encoder, "label_encoder.pkl")


['label_encoder.pkl']